## Imports and preliminaries

In [1]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../src'))
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from recursiveLeastSquares import *
from CalibrationScorers.customResidualCalibrationScorer import customResidualCalibrationScorer
from MultivalidAlgorithms.MultivalidCoverage import multivalid_coverage, eval_fn
from DatasetGeneration.DivisibleDataset import divisible_dataset
from MultivalidAlgorithms.GroupCoverage import group_coverage

In [3]:
def produce_group(feat_index, feat_value):
    '''
        Input: 
            - feat_index: index of desired input feature
            - feat_value: desired value of that feature
        Output:
            - f - function which defines a group; f(x) returns True
                  iff x[feat_index] == feat_value and returns False otherwise.
    '''
    def f(x):
      return (x[feat_index] == feat_value)
    
    return f

# Define group that includes all points
def all_points(x):
    return True

basic_group = [all_points]

# Define 20 overlapping sub-groups - each defined by the value of a single binary feature

twenty_groups = list()
num_groups = 0
for i in range(10):
    for j in range(2):
        curr_group = produce_group(i, j)
        twenty_groups.append(curr_group)
        
group_fn_twenty_groups =(lambda x: [group(x) for group in twenty_groups])

num_groups = len(twenty_groups)

## Synthetic Experiment with groups - Split-Conformal vs. MVP: Single trial

##### Set all parameters for a single trial

In [4]:
# Parameters for our uncertainty quantifier
T = 40000
tau = 0.9
delta = 0.1
num_grid = 100

# Parameters for data generation
x_std = 0.1
y_std = 0.25
d = 100 # choose d > 10

##### Generating data according to ordinary least-squares model - with some groups adding label-noise (of increasing amount) and others not affecting noise.

In [5]:
theta = np.random.normal(loc=np.zeros(d), scale=x_std)

# d-dimension features - first 10 features are binary
xs_binvars = np.random.randint(low = 0, high = 2, size = (T, 10))
xs_remvars = np.random.normal(loc=np.zeros(d - 10), scale=x_std, size=(T, d - 10))
xs = np.concatenate((xs_binvars, xs_remvars), axis = 1)
std_dev_list = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])
std_dev = np.dot(xs_binvars, std_dev_list) + y_std
ys = np.dot(xs, theta) + np.random.normal(loc=0, scale= std_dev, size=T)

x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.5, random_state=42)

In [ ]:
# Separate training data into training (for point-predictor) and calibration
calibration_set_size = 15000
train_set_size = len(y_train) - calibration_set_size
x_train_final = x_train[ : train_set_size]
x_calib = x_train[train_set_size : ]
y_train_final = y_train[ : train_set_size]
y_calib = y_train[train_set_size : ]

In [ ]:
# Train point-predictor using x_train_final and y_train_final
myRLS = RLS(d, 1.0, 1)
for i in range(train_set_size):
    myRLS.add_obs(x_train_final[i], y_train_final[i])

In [ ]:
# Convert y_calib values into threshold values in order to train for conformal prediction 
mult_factor = 250
residualCalibrationScorer = customResidualCalibrationScorer(mult_factor)
residualCalibrationScorer.update(myRLS.predict)

# These are the scores which we want to achieve threshold-calibration with respect to
w_calib = np.array([residualCalibrationScorer.calc_score(x_calib[i], y_calib[i]) for i in range(calibration_set_size)])
w_test = np.array([residualCalibrationScorer.calc_score(x_test[i], y_test[i]) for i in range(len(y_test))])

In [ ]:
# Iterative algorithm for multivalidity 
patches, num_steps = multivalid_coverage(tau=tau, x_train=x_calib, y_train=w_calib, num_grid=num_grid, group_fn=group_fn_twenty_groups)
multivalid_model = lambda x: eval_fn(x, patches=patches, num_grid=num_grid, group_fn=group_fn_twenty_groups)

In [ ]:
# One-shot algorithm for group coverage
group_accurate_model, opt_theta = group_coverage(tau=tau, x_train=x_calib, y_train=w_calib, num_groups=num_groups, group_fn=group_fn_twenty_groups, epochs=75, batch_size = 1024, lr = 0.001)

In [ ]:
from utils.MultivalidPlotting import plot_group_coverage

plot_group_coverage(model=group_accurate_model, tau=tau, x_test=x_test, y_test=w_test, num_groups=num_groups, group_fn=group_fn_twenty_groups, multivalid=False, verbose=True)

In [ ]:
from utils.MultivalidPlotting import plot_pred_set_size

plot_pred_set_size(model=group_accurate_model, tau=tau, x_test=x_test, y_test=w_test, num_groups=num_groups, group_fn=group_fn_twenty_groups, mult_width = mult_factor, multivalid=False, verbose=True)